In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [ ]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(
            base_url = "https://api.fanar.qa/v1",
            api_key  = os.environ["FANAR_API_KEY"],
        )
        # Option A – set a default so you don’t repeat `model=…` later
        client.default_params = {"model": "Fanar-C-1-8.7B"}
        return client    
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="fanar"
deployment = APIKeyManager(model_type, "./azure.env")


In [10]:
class TopicClassifier:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def classify_topic(self, topic, information):
        """
        Classify podcast topic and determine optimal approach
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            
        Returns:
            JSON with classification results
        """
        
        prompt = f"""
أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي طبيعي وجذاب.

المهمة: حلل الموضوع التالي وحدد أفضل نهج لتقديمه في بودكاست عربي.

الموضوع: {topic}
المعلومات الأساسية: {information}

قم بتحليل الموضوع وإرجاع النتيجة بصيغة JSON تحتوي على:

{{
    "primary_category": "الفئة الرئيسية",
    "category_justification": "سبب اختيار هذه الفئة بناءً على طبيعة الموضوع",
    "optimal_style": "الأسلوب الأمثل للمناقشة",
    "discourse_pattern": "نمط الخطاب المناسب",
    "audience_engagement_goal": "هدف تفاعل الجمهور",
    "cultural_sensitivity_level": "مستوى الحساسية الثقافية",
    "controversy_potential": "احتمالية الجدل",
    "key_discussion_angles": [
        "زوايا النقاش الرئيسية المتوقعة",
        "النقاط التي ستثير اهتمام الجمهور العربي"
    ],
    "natural_tension_points": [
        "نقاط التوتر الطبيعية في الموضوع",
        "الجوانب التي قد تثير جدلاً صحياً"
    ],
    "cultural_connection_opportunities": [
        "فرص الربط بالثقافة العربية",
        "المراجع المحلية والإقليمية ذات الصلة"
    ]
}}

الفئات المتاحة:
1. "العلوم والتكنولوجيا" - للمواضيع التقنية والعلمية والابتكارات
2. "السياسة والشؤون العامة" - للمواضيع السياسية والأحداث الجارية والقضايا العامة
3. "القضايا الاجتماعية" - للمواضيع المجتمعية والعلاقات والقيم والتحديات الاجتماعية
4. "الرياضة والترفيه" - للمواضيع الرياضية والفنية والترفيهية
5. "التاريخ والثقافة" - للمواضيع التاريخية والتراثية والثقافية

الأساليب المتاحة:
- "حواري" - حوار طبيعي وودي بين المقدم والضيف
- "تعليمي" - تركيز على الشرح والتعليم بطريقة ممتعة
- "ترفيهي" - مرح وخفيف مع لمسات فكاهية
- "تحليلي" - نقاش عميق ومتخصص وتحليلي

أنماط الخطاب:
- "رسمي" - لغة رسمية ومحترمة
- "ودي" - لغة دافئة ومألوفة
- "جدلي" - نقاش حيوي مع وجهات نظر متعددة
- "سردي" - أسلوب حكواتي وقصصي



مستوى الحساسية الثقافية:
- "عالي" - يتطلب حذراً شديداً في التعامل
- "متوسط" - يحتاج مراعاة ثقافية معتدلة  
- "منخفض" - موضوع مقبول عموماً

احتمالية الجدل:
- "عالية" - موضوع مثير للجدل بطبيعته
- "متوسطة" - قد يثير بعض الاختلافات
- "منخفضة" - موضوع مقبول عموماً

تعليمات مهمة:
- حلل الموضوع بعمق وليس بشكل سطحي
- اعتبر السياق الثقافي العربي في التحليل
- ركز على ما يجعل الموضوع جذاباً للجمهور العربي
- تأكد أن التصنيف يخدم إنتاج محتوى طبيعي وتلقائي
- لا تضع علامات ```json في البداية أو النهاية
- أرجع JSON صحيح فقط بدون أي نص إضافي
-المدة 10 دقائق هي المدة المثلى للحلقة
- "،"  لا تستخدم الفاصلة العربية 
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي احترافي. تخصصك في فهم الجمهور العربي واهتماماته."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent classification
        )
        
        return response.choices[0].message.content

   

In [11]:
# Testing Instructions:

# To test Step 1, add this to a new cell in your notebook:

# Test Step 1: Topic Classification
classifier = TopicClassifier(deployment, "gpt-4o")

# Test with the singlehood topic
topic = "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي"

information = '''
مع انتشار تقنيات الذكاء الاصطناعي بسرعة في العالم العربي، تزداد المخاوف حول تأثيرها على الهوية الثقافية واللغة العربية. 
تشير الدراسات إلى أن 78% من المحتوى الرقمي باللغة الإنجليزية، بينما المحتوى العربي لا يتجاوز 3%. 
معظم نماذج الذكاء الاصطناعي الحالية مدربة على بيانات غربية، مما يثير تساؤلات حول قدرتها على فهم السياق الثقافي العربي.
في المقابل، تسعى دول مثل الإمارات والسعودية لتطوير نماذج ذكاء اصطناعي عربية مثل "جايس" و"الحوراء" لمواجهة هذا التحدي.
التحدي الأكبر يكمن في كيفية الاستفادة من هذه التقنيات لتعزيز الثقافة العربية بدلاً من تهميشها، وضمان أن تخدم الذكاء الاصطناعي قيمنا ومبادئنا.
'''

# Run classification
classification_result = classifier.classify_topic(topic, information)
print("Classification Result:")
print(classification_result)



# Parse and examine the JSON
try:
    parsed_result = json.loads(classification_result)
    # print(f"\nPrimary Category: {parsed_result['primary_category']}")
    # print(f"Optimal Style: {parsed_result['optimal_style']}")
    # print(f"Discourse Pattern: {parsed_result['discourse_pattern']}")
except:
    print("Error parsing JSON result")

Classification Result:
{
    "primary_category": "التاريخ والثقافة",
    "category_justification": "الموضوع يتناول الهوية الثقافية العربية وكيفية الحفاظ عليها في مواجهة التحديات التقنية الحديثة، مما يجعله مرتبطاً بشكل مباشر بالثقافة والتراث.",
    "optimal_style": "تحليلي",
    "discourse_pattern": "جدلي",
    "audience_engagement_goal": "إثارة التفكير والنقاش حول تأثير الذكاء الاصطناعي على الهوية العربية وتشجيع الجمهور على استكشاف الحلول الممكنة.",
    "cultural_sensitivity_level": "عالي",
    "controversy_potential": "متوسطة",
    "key_discussion_angles": [
        "تأثير الذكاء الاصطناعي على اللغة العربية والمحتوى الثقافي.",
        "جهود الدول العربية في تطوير نماذج ذكاء اصطناعي محلية.",
        "كيفية تحقيق التوازن بين الاستفادة من التقنية والحفاظ على الهوية الثقافية.",
        "أهمية تدريب نماذج الذكاء الاصطناعي على بيانات عربية لفهم السياق المحلي."
    ],
    "natural_tension_points": [
        "الهيمنة الغربية على تدريب نماذج الذكاء الاصطناعي وتأثيرها على الثقافة العربية.",
   

In [5]:
class SimplePersonaGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_personas(self, topic, information, classification_result):
        """
        Generate simple but effective host and guest personas
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            
        Returns:
            JSON with simple host and guest personas
        """
        
        # Parse classification to understand the requirements
        try:
            classification = json.loads(classification_result)
        except:
            raise ValueError("Invalid classification JSON provided")
        
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        
        prompt = f"""
أنت خبير في تصميم شخصيات البودكاست العربي.

المهمة: أنشئ مقدم وضيف بسيطين ومناسبين لهذا الموضوع.

الموضوع: {topic}
المعلومات: {information}
الفئة: {primary_category}
الأسلوب المطلوب: {optimal_style}

أرجع النتيجة بصيغة JSON بسيط:

{{
    "host": {{
        "name": "اسم المقدم",
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "guest": {{
        "name": "اسم الضيف", 
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "expertise": "مجال خبرته في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "why_good_match": "لماذا هذا المقدم والضيف مناسبان لهذا الموضوع - جملة واحدة"
}}

متطلبات:
- أسماء عربية مألوفة
- شخصيات بسيطة وقابلة للتصديق
- مناسبة للموضوع والأسلوب المطلوب
- المقدم فضولي والضيف خبير أو صاحب تجربة
- لا تضع علامات ```json
- أرجع JSON فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم شخصيات بودكاست بسيطة ومؤثرة. الأسلوب المطلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )
        
        return response.choices[0].message.content

   

In [ ]:
persona = SimplePersonaGenerator(deployment, "Fanar-C-1-8.7B")
persona_result = persona.generate_personas(topic, information, classification_result)

In [9]:
class ConversationStructureGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_conversation_structure(self, topic, information, classification_result, personas_result):
        """
        Step 3: Generate core conversation structure (V1 skeleton)
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            
        Returns:
            JSON with V1-style conversation structure (without rich dialogue content)
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided for classification or personas")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        discourse_pattern = classification.get("discourse_pattern", "")
        recommended_duration = classification.get("recommended_duration", "10 دقيقة")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
أنت خبير في تصميم هيكل المحادثات للبودكاست العربي.

المهمة: أنشئ الهيكل الأساسي للمحادثة بتنسيق حوار طبيعي.

الموضوع: {topic}
المعلومات: {information}

السياق:
- الفئة: {primary_category}
- الأسلوب: {optimal_style}
- نمط الخطاب: {discourse_pattern}
- المدة: {recommended_duration}

الشخصيات:
- المقدم: {host_name} - {host.get('background', '')}
- الضيف: {guest_name} - {guest.get('background', '')}

أنشئ مخطط المحادثة بصيغة JSON:

{{
    "episode_topic": "{topic}",
    "personas": {{
        "host": {{
            "name": "{host_name}",
            "background": "{host.get('background', '')}",
            "speaking_style": "{host.get('speaking_style', '')}"
        }},
        "guest": {{
            "name": "{guest_name}",
            "background": "{guest.get('background', '')}",
            "speaking_style": "{guest.get('speaking_style', '')}"
        }}
    }},
    "conversation_flow": {{
        "intro1": {{
            "opening_line": "الجملة الافتتاحية الفعلية للمقدم",
            "podcast_introduction": "تعريف بالبودكاست مرتبط بالموضوع",
            "episode_hook": "جملة تشويقية محددة عن الموضوع",
            "tone_guidance": "نبرة مناسبة للأسلوب {optimal_style}"
        }},
        "intro2": {{
            "topic_introduction": "تقديم الموضوع بوضوح",
            "guest_welcome": "ترحيب بالضيف {guest_name}",
            "guest_bio_highlight": "تعريف بخلفية الضيف",
            "transition_to_discussion": "انتقال للنقاش الرئيسي"
        }},
        "main_discussion": [
            {{
                "point_title": "النقطة الرئيسية الأولى للموضوع",
                "personal_angle": "كيف ترتبط بخلفية الشخصيات"
            }},
            {{
                "point_title": "النقطة الرئيسية الثانية للموضوع", 
                "personal_angle": "الزاوية الشخصية لهذه النقطة"
            }},
            {{
                "point_title": "النقطة الرئيسية الثالثة للموضوع",
                "personal_angle": "الربط الشخصي والختامي"
            }}
        ],
        "closing": {{
            "conclusion": {{
                "main_takeaways": "الخلاصات الرئيسية",
                "guest_final_message": "رسالة الضيف الختامية",
                "host_closing_thoughts": "أفكار المقدم الختامية"
            }},
            "outro": {{
                "guest_appreciation": "شكر الضيف",
                "audience_thanks": "شكر المستمعين",
                "call_to_action": "دعوة للتفاعل",
                "final_goodbye": "وداع نهائي"
            }}
        }}
    }},
    "cultural_context": {{
        "proverbs_sayings": [
            "مثل عربي مناسب للموضوع",
            "حكمة ذات صلة بالموضوع"
        ],
        "regional_references": [
            "مرجع محلي متعلق بالموضوع",
            "تجربة عربية ذات صلة"
        ]
    }},
    "language_style": {{
        "formality_level": "مستوى مناسب للأسلوب {optimal_style}",
        "dialect_touches": "لمسات لهجية خفيفة حسب المقدم",
        "vocabulary_richness": "مفردات مناسبة للموضوع"
    }},
    "technical_notes": {{
        "pacing_guidance": "إيقاع مناسب للمدة {recommended_duration}",
        "pause_points": "نقاط توقف طبيعية",
        "emphasis_moments": "لحظات تأكيد مهمة"
    }}
}}

متطلبات مهمة:
- اكتب نصوص فعلية وليس أوصاف
- استخدم أسماء الشخصيات الحقيقية ({host_name}, {guest_name})
- اجعل كل محتوى مرتبط بالموضوع: {topic}
- ركز على بناء الهيكل الأساسي للمحادثة
- اجعل المحتوى جاهز للتطوير في الخطوة التالية
- لا تضع علامات ```json
- أرجع JSON صحيح فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم هيكل المحادثات للبودكاست العربي. الأسلوب المطلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6  # Balanced for structure creation
        )
        
        # Parse and reformat the JSON for proper structure
        try:
            import json
            result_json = json.loads(response.choices[0].message.content)
            return json.dumps(result_json, ensure_ascii=False, indent=2)
        except:
            # If parsing fails, return raw response
            return response.choices[0].message.content

    def validate_conversation_structure(self, structure_json):
        """
        Validate the conversation structure
        """
        required_keys = ["episode_topic", "personas", "conversation_flow", "cultural_context", "language_style", "technical_notes"]
        
        conversation_flow_required = ["intro1", "intro2", "main_discussion", "closing"]
        intro1_required = ["opening_line", "podcast_introduction", "episode_hook"]
        intro2_required = ["topic_introduction", "guest_welcome", "guest_bio_highlight"]
        
        try:
            import json
            structure = json.loads(structure_json)
            
            missing_keys = []
            
            # Check main structure
            for key in required_keys:
                if key not in structure:
                    missing_keys.append(key)
            
            # Check conversation flow
            if "conversation_flow" in structure:
                conv_flow = structure["conversation_flow"]
                for key in conversation_flow_required:
                    if key not in conv_flow:
                        missing_keys.append(f"conversation_flow.{key}")
                
                # Check intro1
                if "intro1" in conv_flow:
                    intro1 = conv_flow["intro1"]
                    for key in intro1_required:
                        if key not in intro1:
                            missing_keys.append(f"intro1.{key}")
                
                # Check intro2
                if "intro2" in conv_flow:
                    intro2 = conv_flow["intro2"]
                    for key in intro2_required:
                        if key not in intro2:
                            missing_keys.append(f"intro2.{key}")
                
                # Check main discussion
                if "main_discussion" in conv_flow:
                    main_disc = conv_flow["main_discussion"]
                    if not isinstance(main_disc, list) or len(main_disc) < 3:
                        missing_keys.append("main_discussion (need at least 3 points)")
                    else:
                        for i, point in enumerate(main_disc):
                            if "point_title" not in point:
                                missing_keys.append(f"main_discussion[{i}].point_title")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Conversation structure validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_structure_quality(self, structure_json):
        """
        Analyze the quality of the conversation structure
        """
        try:
            import json
            structure = json.loads(structure_json)
            
            analysis = {}
            
            # Check completeness
            conv_flow = structure.get("conversation_flow", {})
            analysis["has_intro1"] = bool(conv_flow.get("intro1"))
            analysis["has_intro2"] = bool(conv_flow.get("intro2"))
            analysis["has_main_discussion"] = bool(conv_flow.get("main_discussion"))
            analysis["has_closing"] = bool(conv_flow.get("closing"))
            
            # Check main discussion depth
            main_disc = conv_flow.get("main_discussion", [])
            analysis["discussion_points"] = len(main_disc)
            analysis["adequate_points"] = len(main_disc) >= 3
            
            # Check cultural context
            cultural = structure.get("cultural_context", {})
            analysis["has_proverbs"] = len(cultural.get("proverbs_sayings", [])) >= 1
            analysis["has_regional_refs"] = len(cultural.get("regional_references", [])) >= 1
            
            # Overall readiness
            readiness_indicators = [
                analysis["has_intro1"],
                analysis["has_intro2"],
                analysis["has_main_discussion"],
                analysis["has_closing"],
                analysis["adequate_points"],
                analysis["has_proverbs"]
            ]
            analysis["readiness_score"] = sum(readiness_indicators)
            analysis["ready_for_next_step"] = analysis["readiness_score"] >= 5
            
            return analysis
            
        except:
            return {"error": "Could not analyze structure quality"}

# Usage:
# generator = ConversationStructureGenerator(deployment, "Fanar") 
# structure_result = generator.generate_conversation_structure(topic, information, classification_result, personas_result)

In [ ]:
generator = ConversationStructureGenerator(deployment, "Fanar-C-1-8.7B")
outline_result = generator.generate_conversation_structure(topic, information, classification_result, persona_result)
print(outline_result)

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "سامي الجابري",
      "background": "صحفي مهتم بالتكنولوجيا والثقافة العربية.",
      "speaking_style": "يطرح أسئلة مباشرة ويسعى لتوضيح الأفكار بأسلوب بسيط."
    },
    "guest": {
      "name": "د. ليلى العمري",
      "background": "أستاذة جامعية متخصصة في الذكاء الاصطناعي واللغويات.",
      "speaking_style": "تشرح الأفكار بأسلوب أكاديمي مبسط مدعوم بالأمثلة."
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحباً بكم مستمعينا في بودكاست 'نبض الثقافة'، حيث نناقش القضايا التي تمس هويتنا وثقافتنا في عالم متغير.",
      "podcast_introduction": "اليوم سنتحدث عن موضوع يشغل بال الكثيرين: الذكاء الاصطناعي والهوية العربية، وكيف يمكننا الحفاظ على ثقافتنا في العصر الرقمي.",
      "episode_hook": "مع انتشار الذكاء الاصطناعي، هل يمكن لهذه التقنية أن تصبح حليفاً للثقافة العربية أم أنها تهدد بتهميشها؟"
    },
    "intro2": {
      "topi

In [12]:
class DialogueContentEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def enhance_dialogue_content(self, topic, information, classification_result, personas_result, structure_result):
        """
        Step 4: Enhance conversation structure with rich dialogue content
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            structure_result: JSON string from Step 3 conversation structure
            
        Returns:
            Enhanced structure with rich dialogue content (V1 style)
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
            structure = json.loads(structure_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
أنت خبير في إثراء المحادثات بالمحتوى الحواري الطبيعي للبودكاست العربي.

المهمة: أضف محتوى حواري غني للهيكل الموجود ليصبح جاهزاً لتوليد السكريبت.

الموضوع: {topic}
المعلومات: {information}

السياق:
- الفئة: {primary_category}
- الأسلوب: {optimal_style}
- الحساسية الثقافية: {cultural_sensitivity}

الشخصيات:
- المقدم: {host_name} - {host.get('personality', '')}
- الضيف: {guest_name} - {guest.get('expertise', '')}

الهيكل الحالي:
{structure_result}

أضف العناصر التالية للهيكل الموجود وأرجع النتيجة كاملة:

للـ intro1: أضف
- "spontaneity_elements": [قائمة بعبارات تلقائية محددة]

للـ intro2: أضف  
- "cultural_connections": [ربط ثقافي محدد بالموضوع]

لكل نقطة في main_discussion: أضف
- "spontaneous_triggers": [محفزات تلقائية مرتبطة بالنقطة]
- "disagreement_points": "نقاط اختلاف محتملة"
- "cultural_references": [أمثال ومراجع ثقافية]
- "natural_transitions": "انتقال طبيعي للنقطة التالية"
- "emotional_triggers": "محفزات عاطفية"

أضف قسم جديد:
"spontaneous_moments": {{
    "natural_interruptions": [نقاط تداخل طبيعية],
    "emotional_reactions": [ردود فعل عاطفية],
    "personal_stories": [قصص شخصية حسب خلفية الشخصيات],
    "humorous_moments": [لحظات طريفة مرتبطة بالموضوع]
}},

أضف قسم جديد:
"personality_interactions": {{
    "host_strengths": "نقاط قوة {host_name} في الحوار",
    "guest_expertise": "مجالات خبرة {guest_name}",
    "natural_chemistry": "كيف يتفاعلان طبيعياً",
    "tension_points": "نقاط توتر صحية",
    "collaboration_moments": "لحظات تعاون"
}}

متطلبات مهمة:
- احتفظ بكل المحتوى الموجود من الهيكل الأساسي
- أضف المحتوى الجديد فقط
- اكتب نصوص فعلية وليس أوصاف
- اجعل كل إضافة مرتبطة بالموضوع: {topic}
- اجعل المحتوى مناسب للأسلوب: {optimal_style}
- استخدم أسماء الشخصيات ({host_name}, {guest_name})
- اجعل المحتوى طبيعي وقابل للحوار
- لا تضع علامات ```json
- أرجع JSON كامل محسن
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في إثراء المحادثات بالمحتوى الحواري الطبيعي. الأسلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7  # Higher creativity for dialogue content
        )
        
        # Parse and reformat the JSON for proper structure
        try:
            import json
            result_json = json.loads(response.choices[0].message.content)
            return json.dumps(result_json, ensure_ascii=False, indent=2)
        except:
            # If parsing fails, return raw response
            return response.choices[0].message.content

    def validate_enhanced_content(self, enhanced_json):
        """
        Validate the enhanced dialogue content
        """
        required_new_elements = ["spontaneous_moments", "personality_interactions"]
        
        # Elements that should be added to existing sections
        intro1_should_have = ["spontaneity_elements"]
        intro2_should_have = ["cultural_connections"]
        main_discussion_should_have = ["spontaneous_triggers", "cultural_references", "disagreement_points"]
        
        try:
            import json
            enhanced = json.loads(enhanced_json)
            
            missing_elements = []
            
            # Check new sections
            for element in required_new_elements:
                if element not in enhanced:
                    missing_elements.append(element)
            
            # Check enhanced intro1
            conv_flow = enhanced.get("conversation_flow", {})
            intro1 = conv_flow.get("intro1", {})
            for element in intro1_should_have:
                if element not in intro1:
                    missing_elements.append(f"intro1.{element}")
            
            # Check enhanced intro2
            intro2 = conv_flow.get("intro2", {})
            for element in intro2_should_have:
                if element not in intro2:
                    missing_elements.append(f"intro2.{element}")
            
            # Check enhanced main discussion
            main_disc = conv_flow.get("main_discussion", [])
            for i, point in enumerate(main_disc):
                for element in main_discussion_should_have:
                    if element not in point:
                        missing_elements.append(f"main_discussion[{i}].{element}")
            
            if missing_elements:
                return False, f"Missing enhanced elements: {missing_elements}"
            
            return True, "Dialogue content enhancement validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_enhancement_quality(self, enhanced_json):
        """
        Analyze the quality of dialogue content enhancement
        """
        try:
            import json
            enhanced = json.loads(enhanced_json)
            
            analysis = {}
            
            # Check spontaneous moments
            spont_moments = enhanced.get("spontaneous_moments", {})
            analysis["has_interruptions"] = len(spont_moments.get("natural_interruptions", [])) >= 2
            analysis["has_personal_stories"] = len(spont_moments.get("personal_stories", [])) >= 2
            analysis["has_humor"] = len(spont_moments.get("humorous_moments", [])) >= 1
            
            # Check personality interactions
            personality = enhanced.get("personality_interactions", {})
            analysis["has_chemistry"] = bool(personality.get("natural_chemistry"))
            analysis["has_host_strengths"] = bool(personality.get("host_strengths"))
            analysis["has_guest_expertise"] = bool(personality.get("guest_expertise"))
            
            # Check main discussion enhancement
            conv_flow = enhanced.get("conversation_flow", {})
            main_disc = conv_flow.get("main_discussion", [])
            
            enhanced_points = 0
            for point in main_disc:
                if (point.get("spontaneous_triggers") and 
                    point.get("cultural_references") and 
                    point.get("disagreement_points")):
                    enhanced_points += 1
            
            analysis["enhanced_discussion_points"] = enhanced_points
            analysis["all_points_enhanced"] = enhanced_points == len(main_disc)
            
            # Overall readiness for script generation
            readiness_indicators = [
                analysis["has_interruptions"],
                analysis["has_personal_stories"],
                analysis["has_chemistry"],
                analysis["has_host_strengths"],
                analysis["all_points_enhanced"]
            ]
            analysis["readiness_score"] = sum(readiness_indicators)
            analysis["ready_for_script_generation"] = analysis["readiness_score"] >= 4
            
            return analysis
            
        except:
            return {"error": "Could not analyze enhancement quality"}

# Usage:
# enhancer = DialogueContentEnhancer(deployment, "Fanar")
# enhanced_result = enhancer.enhance_dialogue_content(topic, information, classification_result, personas_result, structure_result)

In [ ]:
style_enhancer = DialogueContentEnhancer(deployment, "Fanar-C-1-8.7B")
style_enhanced_result = style_enhancer.enhance_dialogue_content(topic, information, classification_result, persona_result, outline_result)
print(style_enhanced_result)

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "سامي الجابري",
      "background": "صحفي مهتم بالتكنولوجيا والثقافة العربية.",
      "speaking_style": "يطرح أسئلة مباشرة ويسعى لتوضيح الأفكار بأسلوب بسيط."
    },
    "guest": {
      "name": "د. ليلى العمري",
      "background": "أستاذة جامعية متخصصة في الذكاء الاصطناعي واللغويات.",
      "speaking_style": "تشرح الأفكار بأسلوب أكاديمي مبسط مدعوم بالأمثلة."
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحباً بكم مستمعينا في بودكاست 'نبض الثقافة'، حيث نناقش القضايا التي تمس هويتنا وثقافتنا في عالم متغير.",
      "podcast_introduction": "اليوم سنتحدث عن موضوع يشغل بال الكثيرين: الذكاء الاصطناعي والهوية العربية، وكيف يمكننا الحفاظ على ثقافتنا في العصر الرقمي.",
      "episode_hook": "مع انتشار الذكاء الاصطناعي، هل يمكن لهذه التقنية أن تصبح حليفاً للثقافة العربية أم أنها تهدد بتهميشها؟",
      "spontaneity_elements": [
 

In [14]:
class FinalPolishEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def add_final_polish(self, topic, information, classification_result, personas_result, enhanced_content_result):
        """
        Step 5: Add final polish and spontaneity for natural flow
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            enhanced_content_result: JSON string from Step 4 enhanced content
            
        Returns:
            Final V1-style outline with complete natural flow elements
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
            enhanced_content = json.loads(enhanced_content_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        
        host = personas.get("host", {})
        guest = personas.get("guest", {})
        host_name = host.get('name', 'المقدم')
        guest_name = guest.get('name', 'الضيف')
        
        prompt = f"""
أنت خبير في إضافة اللمسات الأخيرة للمحادثات لجعلها طبيعية وتلقائية تماماً.

المهمة: أضف اللمسات الأخيرة والتدفق الطبيعي للمحتوى المحسن.

الموضوع: {topic}
المعلومات: {information}

السياق:
- الفئة: {primary_category}
- الأسلوب: {optimal_style}
- الحساسية الثقافية: {cultural_sensitivity}

الشخصيات:
- المقدم: {host_name} - {host.get('speaking_style', '')}
- الضيف: {guest_name} - {guest.get('speaking_style', '')}

المحتوى المحسن الحالي:
{enhanced_content_result}

أضف التحسينات النهائية التالية:

1. حسّن "spontaneous_moments" بإضافة المزيد من التفاصيل:
   - أضف المزيد من "natural_interruptions" محددة
   - أضف المزيد من "emotional_reactions" واقعية
   - أضف المزيد من "personal_stories" مناسبة للشخصيات
   - أضف المزيد من "humorous_moments" مرتبطة بالموضوع

2. حسّن "personality_interactions" بتفاصيل أكثر:
   - اجعل "natural_chemistry" أكثر تحديداً
   - أضف تفاصيل لـ "tension_points" 
   - اجعل "collaboration_moments" أكثر وضوحاً

3. أضف قسم جديد "shared_experiences":
   {{
       "common_ground": [
           "نقاط تشابه بين المقدم والضيف",
           "تجارب مشتركة يمكن أن يتحدثا عنها"
       ],
       "generational_perspectives": [
           "اختلافات جيلية قد تظهر في النقاش",
           "وجهات نظر مختلفة بناء على العمر والخبرة"
       ]
   }}

4. أضف قسم جديد "contemporary_relevance":
   {{
       "current_events": [
           "أحداث جارية مرتبطة بالموضوع",
           "تطورات حديثة في المجال"
       ],
       "future_implications": [
           "التأثيرات المستقبلية للموضوع",
           "ما يمكن توقعه في السنوات القادمة"
       ]
   }}

5. حسّن أقسام "cultural_context" بمزيد من التفاصيل:
   - أضف المزيد من "proverbs_sayings"
   - أضف المزيد من "regional_references"
   - اجعل "shared_experiences" أكثر تحديداً

متطلبات نهائية:
- احتفظ بكل المحتوى الموجود من الخطوة 4
- أضف التحسينات النهائية فقط
- اكتب محتوى محدد وليس عام
- اجعل كل إضافة مرتبطة بالموضوع: {topic}
- اجعل الشخصيات تبدو حقيقية ومتفاعلة
- استخدم أسماء الشخصيات ({host_name}, {guest_name})
- اجعل التدفق طبيعي للغاية
- لا تضع علامات ```json
- أرجع JSON كامل ونهائي
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في إضافة اللمسات الأخيرة للمحادثات الطبيعية. الأسلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8  # High creativity for final spontaneity
        )
        
        # Parse and reformat the JSON for proper structure
        try:
            import json
            result_json = json.loads(response.choices[0].message.content)
            return json.dumps(result_json, ensure_ascii=False, indent=2)
        except:
            # If parsing fails, return raw response
            return response.choices[0].message.content

    def validate_final_outline(self, final_json):
        """
        Validate the final polished outline
        """
        required_new_elements = ["shared_experiences", "contemporary_relevance"]
        
        # Enhanced elements that should be richer
        enhanced_elements = {
            "spontaneous_moments": ["natural_interruptions", "emotional_reactions", "personal_stories", "humorous_moments"],
            "personality_interactions": ["natural_chemistry", "tension_points", "collaboration_moments"]
        }
        
        try:
            import json
            final_outline = json.loads(final_json)
            
            missing_elements = []
            
            # Check new sections
            for element in required_new_elements:
                if element not in final_outline:
                    missing_elements.append(element)
            
            # Check enhanced spontaneous moments
            spont_moments = final_outline.get("spontaneous_moments", {})
            for element in enhanced_elements["spontaneous_moments"]:
                if element not in spont_moments:
                    missing_elements.append(f"spontaneous_moments.{element}")
                elif len(spont_moments.get(element, [])) < 2:
                    missing_elements.append(f"spontaneous_moments.{element} (needs at least 2 items)")
            
            # Check enhanced personality interactions
            personality = final_outline.get("personality_interactions", {})
            for element in enhanced_elements["personality_interactions"]:
                if element not in personality:
                    missing_elements.append(f"personality_interactions.{element}")
            
            if missing_elements:
                return False, f"Missing final elements: {missing_elements}"
            
            return True, "Final outline validation successful - ready for V1 script generation"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_final_quality(self, final_json):
        """
        Analyze the quality and completeness of the final outline
        """
        try:
            import json
            final_outline = json.loads(final_json)
            
            analysis = {}
            
            # Check V1 compatibility
            required_v1_sections = ["episode_topic", "personas", "conversation_flow", "spontaneous_moments", "personality_interactions"]
            analysis["v1_compatible"] = all(section in final_outline for section in required_v1_sections)
            
            # Check conversation flow completeness
            conv_flow = final_outline.get("conversation_flow", {})
            analysis["has_complete_flow"] = all(section in conv_flow for section in ["intro1", "intro2", "main_discussion", "closing"])
            
            # Check spontaneous moments richness
            spont_moments = final_outline.get("spontaneous_moments", {})
            spont_count = sum(len(spont_moments.get(key, [])) for key in ["natural_interruptions", "personal_stories", "humorous_moments"])
            analysis["spontaneous_richness"] = spont_count
            analysis["adequate_spontaneity"] = spont_count >= 6
            
            # Check cultural integration depth
            cultural = final_outline.get("cultural_context", {})
            cultural_count = sum(len(cultural.get(key, [])) for key in ["proverbs_sayings", "regional_references", "shared_experiences"])
            analysis["cultural_depth"] = cultural_count
            analysis["adequate_culture"] = cultural_count >= 4
            
            # Check main discussion enhancement
            main_disc = conv_flow.get("main_discussion", [])
            enhanced_points = sum(1 for point in main_disc if all(key in point for key in ["spontaneous_triggers", "cultural_references", "disagreement_points"]))
            analysis["enhanced_discussion_points"] = enhanced_points
            analysis["all_discussions_enhanced"] = enhanced_points == len(main_disc)
            
            # Overall V1 readiness score
            v1_readiness_indicators = [
                analysis["v1_compatible"],
                analysis["has_complete_flow"],
                analysis["adequate_spontaneity"],
                analysis["adequate_culture"],
                analysis["all_discussions_enhanced"]
            ]
            analysis["v1_readiness_score"] = sum(v1_readiness_indicators)
            analysis["ready_for_v1_script_generation"] = analysis["v1_readiness_score"] >= 4
            analysis["perfect_v1_outline"] = analysis["v1_readiness_score"] == 5
            
            return analysis
            
        except:
            return {"error": "Could not analyze final quality"}

    def create_v1_summary(self, final_json):
        """
        Create a summary showing V1 compatibility
        """
        try:
            import json
            final_outline = json.loads(final_json)
            
            summary = {}
            
            # Extract key V1 elements
            conv_flow = final_outline.get("conversation_flow", {})
            summary["intro1_ready"] = bool(conv_flow.get("intro1", {}).get("opening_line"))
            summary["main_points_count"] = len(conv_flow.get("main_discussion", []))
            summary["spontaneous_elements"] = len(final_outline.get("spontaneous_moments", {}).get("natural_interruptions", []))
            summary["cultural_references"] = len(final_outline.get("cultural_context", {}).get("proverbs_sayings", []))
            summary["personality_defined"] = bool(final_outline.get("personality_interactions", {}).get("natural_chemistry"))
            
            # V1 script generation readiness
            summary["v1_script_ready"] = all([
                summary["intro1_ready"],
                summary["main_points_count"] >= 3,
                summary["spontaneous_elements"] >= 2,
                summary["personality_defined"]
            ])
            
            return summary
            
        except:
            return {"error": "Could not create V1 summary"}

# Usage:
# polisher = FinalPolishEnhancer(deployment, "Fanar")
# final_outline = polisher.add_final_polish(topic, information, classification_result, personas_result, enhanced_content_result)

In [ ]:
polisher = FinalPolishEnhancer(deployment, "Fanar-C-1-8.7B")
final_outline = polisher.add_final_polish(topic, information, classification_result, persona_result, style_enhanced_result)
print(final_outline)

{
  "episode_topic": "الذكاء الاصطناعي والهوية العربية: كيف نحافظ على ثقافتنا في العصر الرقمي",
  "personas": {
    "host": {
      "name": "سامي الجابري",
      "background": "صحفي مهتم بالتكنولوجيا والثقافة العربية.",
      "speaking_style": "يطرح أسئلة مباشرة ويسعى لتوضيح الأفكار بأسلوب بسيط."
    },
    "guest": {
      "name": "د. ليلى العمري",
      "background": "أستاذة جامعية متخصصة في الذكاء الاصطناعي واللغويات.",
      "speaking_style": "تشرح الأفكار بأسلوب أكاديمي مبسط مدعوم بالأمثلة."
    }
  },
  "conversation_flow": {
    "intro1": {
      "opening_line": "مرحباً بكم مستمعينا في بودكاست 'نبض الثقافة'، حيث نناقش القضايا التي تمس هويتنا وثقافتنا في عالم متغير.",
      "podcast_introduction": "اليوم سنتحدث عن موضوع يشغل بال الكثيرين: الذكاء الاصطناعي والهوية العربية، وكيف يمكننا الحفاظ على ثقافتنا في العصر الرقمي.",
      "episode_hook": "مع انتشار الذكاء الاصطناعي، هل يمكن لهذه التقنية أن تصبح حليفاً للثقافة العربية أم أنها تهدد بتهميشها؟",
      "spontaneity_elements": [
 

In [17]:
class HybridScriptGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment
        
        # V1-style rich dialogue examples
        self.arabic_dialogue_styles = {
            "حواري": {
                "host_example": "أحمد: يا أهلاً نور! كيف الحال؟ اممم... قوليلي، شو اللي خلاكِ تدخلي هذا المجال؟ <happy>",
                "guest_example": "نور: أهلاً أحمد! الله يعطيك العافية... يعني بصراحة، هاي قصة طويلة شوي [pause: 2s] بس باختصار، كنت أشوف المشاكل حولي وأقول: ليش ما نحلها بالتقنية؟"
            },
            "تعليمي": {
                "host_example": "أحمد: اممم... نور، ممكن تشرحي لنا بطريقة بسيطة، يعني شلون تشتغل هاي التقنية؟ <happy>",
                "guest_example": "نور: طبعاً أحمد! يعني... اههه كيف أشرح [pause: 2s] تخيل إنك عندك نظام ذكي جداً، بس هذا النظام مش إنسان، هو كمبيوتر! واو صح؟"
            },
            "ترفيهي": {
                "host_example": "أحمد: هاي نور! <happy> قوليلي، إيش أغرب موقف صار معكِ في الشغل؟ يعني شي يضحك؟",
                "guest_example": "نور: ههههه واو أحمد! بصراحة مواقف كثيرة... اممم مرة كنت أجرب البرنامج وفجأة [pause: 2s] خلاص ما عاد يشتغل! قعدت أصرخ: وين راح كودي؟! <surprise>"
            },
            "تحليلي": {
                "host_example": "أحمد: نور، بناءً على الإحصائيات الحديثة، وش رايكِ في التحديات الرئيسية اللي تواجه هذا المجال؟",
                "guest_example": "نور: سؤال ممتاز أحمد... يعني إذا نتكلم بشكل تحليلي، عندنا ثلاث تحديات أساسية [pause: 2s] أولها التقنية، ثانيها التمويل، وثالثها... اممم التقبل المجتمعي"
            }
        }
        
        # V1-style comprehensive fillers guide
        self.fillers_guide = """
استخدم حشو المحادثة الطبيعي بكثافة متوسطة:
- تفكير: اممم، اههه، يعني كيف أقول، خلاص، شوف
- تأكيد: طبعاً، تماماً، بالضبط، صحيح، أكيد
- تردد: يعنييييي، يعني، اه ما أدري، مش عارف
- انفعال: واو، يا الله، ما شاء الله، الله يعطيك العافية
- ربط: بس، لكن، وبعدين، يا أخي، اسمع، انتبه
- خليجي خفيف: شلون، وش رايك، زين، ماشي الحال، الله يعافيك
"""

    def generate_enhanced_intro(self, topic, final_outline_result, optimal_style):
        """
        Step 1: Generate enhanced intro (intro1 + intro2) - V1 enhancer style
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract outline elements
        conv_flow = outline.get("conversation_flow", {})
        intro1 = conv_flow.get("intro1", {})
        intro2 = conv_flow.get("intro2", {})
        personas = outline.get("personas", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style examples
        style_examples = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])
        
        prompt = f"""
أنت خبير في كتابة مقدمات البودكاست العربي الطبيعية والجذابة.

المهمة: اكتب مقدمة كاملة طبيعية (intro1 + intro2) تجمع بين الترحيب وتقديم الموضوع والضيف.

أمثلة على الأسلوب المطلوب:
المقدم: {style_examples["host_example"]}
الضيف: {style_examples["guest_example"]}

عناصر المقدمة من المخطط:
الافتتاحية: {intro1.get('opening_line', '')}
تعريف البودكاست: {intro1.get('podcast_introduction', '')}
جملة التشويق: {intro1.get('episode_hook', '')}
العناصر التلقائية: {intro1.get('spontaneity_elements', [])}

تقديم الموضوع: {intro2.get('topic_introduction', '')}
ترحيب بالضيف: {intro2.get('guest_welcome', '')}
تعريف الضيف: {intro2.get('guest_bio_highlight', '')}
الروابط الثقافية: {intro2.get('cultural_connections', [])}

المراجع الثقافية المتاحة: {cultural_context.get('proverbs_sayings', [])}

{self.fillers_guide}

متطلبات المقدمة:
1. ابدأ بالمقدم {host_name} يتحدث منفرداً (intro1)
2. ثم أدخل الضيف {guest_name} في المحادثة (intro2)
3. استخدم 70% فصحى و 30% لمسات خليجية خفيفة
4. أضف حشو طبيعي متوسط الكثافة
5. أدرج العناصر التلقائية والثقافية بطبيعية
6. اجعل الانتقال سلس من المقدم لوحده إلى حوار مع الضيف
7. استخدم أسماء الشخصيات الفعلية
8. أضف تفاعلات طبيعية: <happy>, <pause: 2s>, <overlap>
9. اجعل المقدمة مشوقة ومدتها 2-3 دقائق
10. انته بانتقال طبيعي للنقاش الرئيسي

تنسيق الحوار:
المقدم: [النص]
الضيف: [النص]

اكتب المقدمة كاملة:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في كتابة مقدمات بودكاست عربية طبيعية وجذابة. الأسلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        
        return response.choices[0].message.content

    def generate_flowing_main_discussion(self, topic, final_outline_result, optimal_style, intro_dialogue):
        """
        Step 2: Generate flowing main discussion (all 3 points) - V1 dialogue generator style
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract outline elements
        conv_flow = outline.get("conversation_flow", {})
        main_discussion = conv_flow.get("main_discussion", [])
        personas = outline.get("personas", {})
        spontaneous_moments = outline.get("spontaneous_moments", {})
        personality_interactions = outline.get("personality_interactions", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style examples
        style_examples = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])
        
        # Build main points summary
        points_summary = ""
        for i, point in enumerate(main_discussion):
            points_summary += f"""
النقطة {i+1}: {point.get('point_title', '')}
- الزاوية الشخصية: {point.get('personal_angle', '')}
- المحفزات التلقائية: {point.get('spontaneous_triggers', [])}
- نقاط الاختلاف: {point.get('disagreement_points', '')}
- المراجع الثقافية: {point.get('cultural_references', [])}
- المحفزات العاطفية: {point.get('emotional_triggers', '')}
"""
        
        prompt = f"""
أنت خبير في كتابة الحوارات الطبيعية المتدفقة للبودكاست العربي.

المهمة: اكتب النقاش الرئيسي كحوار متدفق طبيعي يغطي جميع النقاط الثلاث.

أمثلة على الأسلوب المطلوب:
المقدم: {style_examples["host_example"]}
الضيف: {style_examples["guest_example"]}

السياق - نهاية المقدمة:
{intro_dialogue[-300:]}

النقاط المطلوب تغطيتها:
{points_summary}

عناصر التلقائية المتاحة:
- المقاطعات الطبيعية: {spontaneous_moments.get('natural_interruptions', [])}
- ردود الفعل العاطفية: {spontaneous_moments.get('emotional_reactions', [])}
- القصص الشخصية: {spontaneous_moments.get('personal_stories', [])}
- اللحظات المرحة: {spontaneous_moments.get('humorous_moments', [])}

ديناميكية الشخصيات:
- الكيمياء الطبيعية: {personality_interactions.get('natural_chemistry', '')}
- نقاط التوتر: {personality_interactions.get('tension_points', '')}
- لحظات التعاون: {personality_interactions.get('collaboration_moments', '')}

{self.fillers_guide}

متطلبات النقاش:
1. ابدأ بانتقال طبيعي من المقدمة
2. غط النقاط الثلاث بحوار متدفق وطبيعي
3. لا تلتزم بترتيب النقاط بصرامة - دع الحوار يتدفق
4. أدرج العناصر التلقائية والثقافية بطبيعية
5. أضف نقاط اختلاف وجدل صحي
6. استخدم القصص الشخصية والمراجع الثقافية
7. اجعل {host_name} و{guest_name} يتفاعلان بشخصيتهما
8. أضف لحظات مرحة ومفاجئة
9. استخدم 70% فصحى و 30% لمسات خليجية
10. المدة: 6-7 دقائق من الحوار الطبيعي
11. انته بتمهيد طبيعي للختام

تنسيق الحوار:
المقدم: [النص]
الضيف: [النص]

اكتب النقاش كاملاً:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في كتابة حوارات متدفقة وطبيعية. الأسلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        
        return response.choices[0].message.content

    def generate_natural_closing(self, topic, final_outline_result, optimal_style, intro_dialogue, main_dialogue):
        """
        Step 3: Generate natural closing (conclusion + outro) - V1 enhancer style
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Extract outline elements
        conv_flow = outline.get("conversation_flow", {})
        closing = conv_flow.get("closing", {})
        personas = outline.get("personas", {})
        cultural_context = outline.get("cultural_context", {})
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        
        # Get style examples
        style_examples = self.arabic_dialogue_styles.get(optimal_style, self.arabic_dialogue_styles["تعليمي"])
        
        prompt = f"""
أنت خبير في كتابة خواتيم البودكاست العربي المؤثرة والطبيعية.

المهمة: اكتب ختام طبيعي ومؤثر (خلاصة + وداع) للحلقة.

أمثلة على الأسلوب المطلوب:
المقدم: {style_examples["host_example"]}

السياق - نهاية النقاش:
{main_dialogue[-300:]}

عناصر الختام من المخطط:
الخلاصات المهمة: {closing.get('conclusion', {}).get('main_takeaways', '')}
رسالة الضيف الختامية: {closing.get('conclusion', {}).get('guest_final_message', '')}
أفكار المقدم الختامية: {closing.get('conclusion', {}).get('host_closing_thoughts', '')}

شكر الضيف: {closing.get('outro', {}).get('guest_appreciation', '')}
شكر المستمعين: {closing.get('outro', {}).get('audience_thanks', '')}
دعوة للتفاعل: {closing.get('outro', {}).get('call_to_action', '')}
الوداع النهائي: {closing.get('outro', {}).get('final_goodbye', '')}

المراجع الثقافية: {cultural_context.get('proverbs_sayings', [])}

{self.fillers_guide}

متطلبات الختام:
1. ابدأ بانتقال طبيعي من النقاش
2. لخص النقاط المهمة بطريقة طبيعية وليس آلية
3. اجعل {guest_name} يقدم رسالة ختامية مؤثرة
4. اجعل {host_name} يضيف أفكاره الختامية
5. أضف شكر صادق ودافئ للضيف والمستمعين
6. أدرج دعوة طبيعية للتفاعل
7. استخدم لمسة ثقافية في الوداع
8. اجعل النهاية تترك أثراً إيجابياً
9. استخدم 70% فصحى و 30% لمسات خليجية
10. المدة: 1-2 دقيقة مؤثرة

تنسيق الحوار:
المقدم: [النص]
الضيف: [النص]

اكتب الختام كاملاً:
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في كتابة خواتيم مؤثرة وطبيعية. الأسلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        
        return response.choices[0].message.content

    def generate_complete_script(self, topic, final_outline_result):
        """
        Generate complete script using 3-step hybrid approach
        """
        
        import json
        try:
            outline = json.loads(final_outline_result)
        except:
            raise ValueError("Invalid outline JSON")
        
        # Determine optimal style from outline or default
        optimal_style = "تعليمي"  # Default
        if "language_style" in outline:
            formality = outline["language_style"].get("formality_level", "")
            if "حواري" in formality or "ودي" in formality:
                optimal_style = "حواري"
            elif "تحليلي" in formality:
                optimal_style = "تحليلي"
            elif "مرح" in formality or "ترفيهي" in formality:
                optimal_style = "ترفيهي"
        
        print(f"🎙️ بدء توليد السكريبت بأسلوب: {optimal_style}")
        print("=" * 50)
        
        # Step 1: Enhanced Intro
        print("📝 الخطوة 1: توليد المقدمة المحسنة...")
        intro_dialogue = self.generate_enhanced_intro(topic, final_outline_result, optimal_style)
        print("✅ تم إنجاز المقدمة")
        
        # Step 2: Flowing Main Discussion
        print("📝 الخطوة 2: توليد النقاش المتدفق...")
        main_dialogue = self.generate_flowing_main_discussion(topic, final_outline_result, optimal_style, intro_dialogue)
        print("✅ تم إنجاز النقاش الرئيسي")
        
        # Step 3: Natural Closing
        print("📝 الخطوة 3: توليد الختام الطبيعي...")
        closing_dialogue = self.generate_natural_closing(topic, final_outline_result, optimal_style, intro_dialogue, main_dialogue)
        print("✅ تم إنجاز الختام")
        
        # Combine all parts
        complete_script = f"""=== المقدمة ===
{intro_dialogue}

=== النقاش الرئيسي ===
{main_dialogue}

=== الختام ===
{closing_dialogue}"""
        
        print("🎉 تم إنجاز السكريبت الكامل!")
        
        return {
            "intro": intro_dialogue,
            "main_discussion": main_dialogue,
            "closing": closing_dialogue,
            "complete_script": complete_script,
            "script_length": len(complete_script),
            "estimated_duration": "10 دقائق تقريباً",
            "style_used": optimal_style
        }

# Usage:
# generator = HybridScriptGenerator(deployment, "Fanar")
# script_result = generator.generate_complete_script(topic, final_outline_result)

In [ ]:
script_generator = HybridScriptGenerator(deployment, "Fanar-C-1-8.7B")
script_result = script_generator.generate_complete_script(topic, final_outline)
print(script_result["complete_script"])

🎙️ بدء توليد السكريبت بأسلوب: تحليلي
📝 الخطوة 1: توليد المقدمة المحسنة...
✅ تم إنجاز المقدمة
📝 الخطوة 2: توليد النقاش المتدفق...
✅ تم إنجاز النقاش الرئيسي
📝 الخطوة 3: توليد الختام الطبيعي...
✅ تم إنجاز الختام
🎉 تم إنجاز السكريبت الكامل!
=== المقدمة ===
المقدم: مرحباً بكم مستمعينا الكرام، في حلقة جديدة من بودكاست "نبض الثقافة"، حيث نغوص في أعماق القضايا التي تلامس هويتنا وثقافتنا في عالم يتغير بسرعة. أنا سامي الجابري، سعيد جداً أنكم معنا اليوم، ويا أخي، حلقتنا اليوم فعلاً مثيرة. يعني شوفوا، الذكاء الاصطناعي صار جزءاً من حياتنا اليومية، من التطبيقات اللي نستخدمها على الهاتف إلى الأدوات اللي بدأت تغيّر مسار الأعمال. بس، السؤال اللي يطرح نفسه: وش تأثير هذا كله على هويتنا العربية؟ <pause: 2s> 

المقدم: زين، يعني إذا فكرتوا شوي، هل الذكاء الاصطناعي ممكن يكون داعماً للثقافة العربية؟ أو بالعكس، يمكن يكون تهديد؟ أنا شخصياً دايماً أتساءل: لو الذكاء الاصطناعي صار يتحدث باللهجات المحلية مثلاً، كيف بيكون الحوار؟ وهل فعلاً بيعكس عمق حضارتنا؟ طبعاً الموضوع هذا كبير، وما نقدر نناقشه بدون خبرة عميقة. 
